# 🔗 Workflow Integration - Complete SAR Processing System

Welcome to Phase 4 of the Financial Services Agentic AI Project!

In this notebook, you'll integrate both AI agents into a complete **end-to-end SAR processing workflow** that demonstrates real-world financial compliance automation.

## 🎯 Learning Objectives
- Build a complete two-stage AI workflow with human oversight
- Implement human-in-the-loop decision gates for compliance
- Generate complete SAR documents from AI analysis
- Create comprehensive audit trails for regulatory examination
- Demonstrate cost optimization through intelligent agent coordination

## 📋 Business Context
This workflow simulates how banks actually process suspicious activity reports:
1. **Risk Screening**: AI agents analyze transaction patterns for suspicious activity
2. **Human Review**: Compliance officers review AI findings before proceeding
3. **Narrative Generation**: Only approved cases get full compliance documentation
4. **SAR Filing**: Complete regulatory forms are generated for submission
5. **Audit Documentation**: Every decision is logged for regulatory examination

## 🏗️ System Architecture

```
📊 CSV Data → 🔍 Risk Analyst → 👤 Human Decision → ✅ Compliance Officer → 📄 SAR Document
              (Chain-of-Thought)    (Gate)         (ReACT Framework)     (FinCEN Ready)
```

## 🚀 Prerequisites Check

Before starting, ensure you have completed:
- ✅ Phase 1: Foundation components (`foundation_sar.py`)
- ✅ Phase 2: Risk Analyst Agent (`risk_analyst_agent.py`)
- ✅ Phase 3: Compliance Officer Agent (`compliance_officer_agent.py`)
- ✅ Both agents pass their comprehensive test scenarios

If any component is missing, return to previous notebooks to complete implementation.

In [1]:
# Setup and Environment Configuration
import os
import sys
import json
import pandas as pd
import uuid
import hashlib
from datetime import datetime, timedelta
from dotenv import load_dotenv

# Add src directory to Python path for module imports
sys.path.append(os.path.abspath('../src'))

# Load environment variables
load_dotenv('../.env')

print("📚 Libraries imported successfully!")
print("🔐 Environment variables loaded")
print("📂 Source directory added to Python path")

📚 Libraries imported successfully!
🔐 Environment variables loaded
📂 Source directory added to Python path


In [ ]:
# OpenAI Setup for Vocareum
import openai

# Initialize OpenAI client for Vocareum
openai_api_key = os.getenv('OPENAI_VOCAREUM_API_KEY')

if not openai_api_key:
    print("⚠️ WARNING: No OpenAI API key found!")
    print("Please set OPENAI_API_KEY in your .env file")
    print("Get your Vocareum OpenAI API key from 'Cloud Resources' in your workspace")
else:
    # Vocareum requires routing through their servers
    client = openai.OpenAI(
        base_url="https://openai.vocareum.com/v1",
        api_key=openai_api_key
    )
    print("✅ OpenAI client initialized with Vocareum routing")
    print(f"🔑 API key: {openai_api_key[:8]}...{openai_api_key[-4:]}")
    print("📍 Base URL: https://openai.vocareum.com/v1")

✅ OpenAI client initialized with Vocareum routing
🔑 API key: sk-proj-...x7IA
📍 Base URL: https://openai.vocareum.com/v1


In [3]:
# TODO: Import Your Implemented Components
# Students: Import your foundation components and agents

print("📋 TODO: Import your implemented components")
print("Uncomment and modify these imports once you've implemented all components:")

from foundation_sar import (
     CustomerData,
     AccountData,
     TransactionData,
     CaseData,
     RiskAnalystOutput,
     ComplianceOfficerOutput,
     ExplainabilityLogger,
     DataLoader,
     load_csv_data
)
from risk_analyst_agent import RiskAnalystAgent
from compliance_officer_agent import ComplianceOfficerAgent

# TODO: Create agent instances
explainability_logger = ExplainabilityLogger("../outputs/audit_logs/workflow_integration.jsonl")
risk_agent = RiskAnalystAgent(client, explainability_logger)
compliance_agent = ComplianceOfficerAgent(client, explainability_logger)

print("✅ Ready to import components after implementation")

📋 TODO: Import your implemented components
Uncomment and modify these imports once you've implemented all components:
✅ Ready to import components after implementation


## 📊 Step 1: Data Loading and Preprocessing

Load the financial data and prepare it for analysis.

In [ ]:
# TODO: Load and Preprocess Financial Data
# Students: Load customer, account, and transaction data

def load_and_preprocess_data():
    """
    It loads CSV data, handles null values, and converts them to dictionaries.
    """
    print("📊 Loading Financial Data...")
    
    try:
        # Load using pandas
        customers_df = pd.read_csv("../data/customers.csv", dtype={'ssn_last_4': str})
        accounts_df = pd.read_csv("../data/accounts.csv")
        transactions_df = pd.read_csv("../data/transactions.csv")
        
        # Handling NaNs (Pydantic validator 'blank_to_none' helps, but it's good to be sure)
        customers_df = customers_df.fillna('')
        # Converts numeric columns that may have come as string/NaN.
        customers_df['annual_income'] = pd.to_numeric(customers_df['annual_income'], errors='coerce').fillna(0.0)
        
        # Accounts
        accounts_df = accounts_df.fillna('')
        accounts_df['current_balance'] = pd.to_numeric(accounts_df['current_balance'], errors='coerce').fillna(0.0)
        accounts_df['average_monthly_balance'] = pd.to_numeric(accounts_df['average_monthly_balance'], errors='coerce').fillna(0.0)

        # Transactions
        transactions_df = transactions_df.fillna('')
        transactions_df['amount'] = pd.to_numeric(transactions_df['amount'], errors='coerce').fillna(0.0)
        
        # Convert DataFrames to list of dicts
        customers_data = customers_df.to_dict('records')
        accounts_data = accounts_df.to_dict('records')
        transactions_data = transactions_df.to_dict('records')
        
        print(f"📈 Loaded: {len(customers_data)} customers, {len(accounts_data)} accounts, {len(transactions_data)} transactions")
        return customers_data, accounts_data, transactions_data

    except Exception as e:
        print(f"❌ Erro ao carregar dados: {e}")
        return [], [], []

# Load data
customers_data, accounts_data, transactions_data = load_and_preprocess_data()

📊 Loading Financial Data...
📈 Loaded: 150 customers, 178 accounts, 4268 transactions


## 🎯 Step 2: Customer Risk Screening

Implement intelligent customer screening to identify high-risk cases for detailed analysis.

In [ ]:
# TODO: Implement Customer Risk Screening
# Students: Create risk-based customer screening logic

def screen_high_risk_customers(customers_data, accounts_data, transactions_data, top_n=5):
    """
    Filtra clientes de alto risco baseados em regras determinísticas.
    """
    print("🔍 Customer Risk Screening...")
    selected_customers = []
    
    for customer in customers_data:
        c_id = customer['customer_id']
        
        # 1. Aggregates customer accounts
        cust_accounts = [acc for acc in accounts_data if acc['customer_id'] == c_id]
        acc_ids = {acc['account_id'] for acc in cust_accounts}
        
        # 2. Aggregates transactions from accounts
        cust_txns = [txn for txn in transactions_data if txn['account_id'] in acc_ids]
        
        # 3. Calculates risk indicators
        total_amount = sum(abs(t['amount']) for t in cust_txns)
        txn_count = len(cust_txns)
        risk_rating = customer.get('risk_rating', 'Low')
        
        risk_flags = []
        
        # Screening Rules
        if risk_rating in ['High', 'Medium']:
            risk_flags.append('high_risk_rating')
        if total_amount > 100000:
            risk_flags.append('large_volume')
        if txn_count > 50:
            risk_flags.append('high_frequency')
        
        # If there's at least 1 flag (i.e., it's worth the AI to analyze), add to list
        if len(risk_flags) >= 1:
            selected_customers.append({
                'customer': customer,
                'accounts': cust_accounts,
                'transactions': cust_txns,
                'score_metrics': {
                    'total_amount': total_amount,
                    'flags': risk_flags
                }
            })
            
    # Sorts: Prioritize those with more flags and higher financial volume
    selected_customers.sort(
        key=lambda x: (len(x['score_metrics']['flags']), x['score_metrics']['total_amount']), 
        reverse=True
    )
    
    final_selection = selected_customers[:top_n]
    print(f"📊 Selected {len(final_selection)} high-risk customers for detailed AI analysis.")
    return final_selection

# Execute the screening
selected_customers = screen_high_risk_customers(customers_data, accounts_data, transactions_data)

🔍 Customer Risk Screening...
📊 Selected 5 high-risk customers for detailed AI analysis.


## 🤖 Step 3: Two-Stage AI Analysis with Human Gates

Implement the core two-stage workflow:
1. **Stage 1**: Risk Analyst performs Chain-of-Thought analysis
2. **Human Gate**: Review and decision to proceed
3. **Stage 2**: Compliance Officer generates ReACT narratives (only if approved)

In [ ]:
# TODO: Implement Two-Stage AI Workflow
# Students: Build the complete workflow with human decision gates

def run_two_stage_sar_workflow(selected_customers):
    print("🤖 Starting Two-Stage SAR Processing Workflow...")
    
    processed_cases = []
    approved_sars = []
    rejected_cases = []
    audit_decisions = []
    
    loader = DataLoader(explainability_logger)

    for i, customer_bundle in enumerate(selected_customers, 1):
        cust_data = customer_bundle['customer']
        print(f"\n[{i}/{len(selected_customers)}] Processing Customer: {cust_data['name']} (ID: {cust_data['customer_id']})")
        print("=" * 60)
        
        try:
            # 1. Create a CaseData object (unifying the data)
            case_data = loader.create_case_from_data(
                cust_data,
                customer_bundle['accounts'],
                customer_bundle['transactions']
            )
            processed_cases.append(case_data)
            
            # ---------------------------------------------------------
            # STAGE 1: Risk Analysis (Chain-of-Thought)
            # ---------------------------------------------------------
            print("🔍 Running Risk Analyst Agent...")
            risk_analysis = risk_agent.analyze_case(case_data)
            
            # Displays results for Humans
            print(f"\n--- AI FINDINGS ---")
            print(f"Classification: {risk_analysis.classification}")
            print(f"Risk Level:     {risk_analysis.risk_level}")
            print(f"Confidence:     {risk_analysis.confidence_score:.2f}")
            print(f"Reasoning:      {risk_analysis.reasoning[:150]}...")
            
            # ---------------------------------------------------------
            # HUMAN DECISION GATE
            # ---------------------------------------------------------
            # Note: In production, this would be a web interface. Here we use input().
            # If the score is too low, we could auto-reject, but the prompt asks for human decision.
            
            decision = input(">>> Proceed with SAR filing? (yes/no): ").strip().lower()
            should_proceed = decision in ['yes', 'y']
            
            if should_proceed:
                print("✅ Approved. Proceeding to Compliance Narrative generation.")
                
                # -----------------------------------------------------
                # STAGE 2: Compliance Officer (ReACT)
                # -----------------------------------------------------
                compliance_review = compliance_agent.generate_compliance_narrative(case_data, risk_analysis)
                
                # Generates Final SAR Document
                sar_document = create_sar_document(case_data, risk_analysis, compliance_review)
                save_sar_document(sar_document)
                
                approved_sars.append(sar_document)
                print(f"📄 SAR Generated: {sar_document['sar_metadata']['sar_id']}")
                
            else:
                print("❌ Rejected by human reviewer.")
                rejected_cases.append({
                    'case_id': case_data.case_id,
                    'reason': 'Human Rejection'
                })

            # Log for Audit
            audit_decisions.append({
                'case_id': case_data.case_id,
                'customer_name': cust_data['name'],
                'ai_risk_level': risk_analysis.risk_level,
                'human_decision': 'APPROVED' if should_proceed else 'REJECTED',
                'timestamp': datetime.now().isoformat()
            })
            
        except Exception as e:
            print(f"❌ Error processing case: {e}")
            import traceback
            traceback.print_exc()

    return processed_cases, approved_sars, rejected_cases, audit_decisions

## 📄 Step 4: SAR Document Generation

Create complete, FinCEN-ready SAR documents with all required metadata.

In [ ]:
# TODO: Implement SAR Document Generation
# Students: Create complete SAR documents for regulatory submission

def create_sar_document(case_data, risk_analysis, compliance_review):
    """
    Cria o documento JSON final do SAR combinando todas as saídas.
    """
    sar_id = f"SAR-{uuid.uuid4().hex[:8].upper()}"
    filing_date = datetime.now().strftime("%Y-%m-%d")
    
    sar_doc = {
        "sar_metadata": {
            "sar_id": sar_id,
            "filing_date": filing_date,
            "filing_type": "Initial",
            "fincen_form_type": "111",
            "status": "Ready for Submission"
        },
        "subject_information": {
            "name": case_data.customer.name,
            "ssn_last_4": case_data.customer.ssn_last_4,
            "address": case_data.customer.address,
            "occupation": getattr(case_data.customer, 'occupation', 'Unknown'),
            "birth_date": case_data.customer.date_of_birth
        },
        "suspicious_activity": {
            "classification": risk_analysis.classification,
            "date_range_start": min([t.transaction_date for t in case_data.transactions], default=""),
            "date_range_end": max([t.transaction_date for t in case_data.transactions], default=""),
            "total_amount": sum([t.amount for t in case_data.transactions]),
            "key_indicators": risk_analysis.key_indicators
        },
        "narrative": {
            "text": compliance_review.narrative,
            "reasoning": compliance_review.narrative_reasoning,
            "regulatory_citations": compliance_review.regulatory_citations,
            "word_count": len(compliance_review.narrative.split())
        }
    }
    return sar_doc

def save_sar_document(sar_document):
    """Salva o SAR em disco."""
    output_dir = "../outputs/filed_sars"
    os.makedirs(output_dir, exist_ok=True)
    
    filename = f"{output_dir}/{sar_document['sar_metadata']['sar_id']}.json"
    
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(sar_document, f, indent=2, ensure_ascii=False)
    
    # print(f"💾 Saved to {filename}")

print("📄 Funções de geração de SAR definidas.")

📄 Funções de geração de SAR definidas.


In [ ]:
processed_cases, approved_sars, rejected_cases, audit_decisions = run_two_stage_sar_workflow(selected_customers)

🤖 Starting Two-Stage SAR Processing Workflow...

[1/5] Processing Customer: Jacqueline Rodriguez (ID: CUST_0111)
🔍 Running Risk Analyst Agent...

--- AI FINDINGS ---
Classification: Money_Laundering
Risk Level:     High
Confidence:     0.85
Reasoning:      The customer has a high risk rating and is involved in high volume and high value transactions, including large incoming and outgoing wire transfers. ...
✅ Approved. Proceeding to Compliance Narrative generation.
🔍 DEBUG: Starting narrative generation...
📄 SAR Generated: SAR-CFAA9F63

[2/5] Processing Customer: Michael Stanley (ID: CUST_0062)
🔍 Running Risk Analyst Agent...

--- AI FINDINGS ---
Classification: Money_Laundering
Risk Level:     High
Confidence:     0.90
Reasoning:      The customer has a medium risk rating and has been involved in a series of large, rapid, and unusual wire transfers. This includes a large incoming wi...
✅ Approved. Proceeding to Compliance Narrative generation.
🔍 DEBUG: Starting narrative generation...

## 📊 Step 5: Workflow Metrics and Analysis

Analyze the efficiency and effectiveness of your AI-powered SAR processing system.

In [ ]:
# TODO: Implement Workflow Analysis and Metrics
# Students: Calculate efficiency metrics and cost analysis

def analyze_workflow_efficiency(processed_cases, approved_sars, rejected_cases, audit_decisions):
    print("\n📊 WORKFLOW EFFICIENCY REPORT")
    print("=" * 40)
    
    total = len(processed_cases)
    if total == 0:
        print("No cases processed.")
        return

    approved = len(approved_sars)
    rejected = len(rejected_cases)
    
    print(f"Total Cases Screened (AI Stage 1): {total}")
    print(f"Human Approved (Proceeded to Stage 2): {approved}")
    print(f"Human Rejected (Stopped at Stage 1): {rejected}")
    
    approval_rate = (approved / total) * 100
    print(f"Approval Rate: {approval_rate:.1f}%")
    
    # Cost Estimation
    # Assuming: Stage 1 = $0.05, Stage 2 = $0.15 (due to larger/more complex output)
    cost_stage_1 = total * 0.05
    cost_stage_2 = approved * 0.15
    total_cost = cost_stage_1 + cost_stage_2
    
    # Cost if we ran everything in single-stage (all generate narrative)
    theoretical_cost = total * (0.05 + 0.15)
    savings = theoretical_cost - total_cost
    
    print(f"\n💰 COST OPTIMIZATION")
    print(f"Actual API Cost (Two-Stage): ${total_cost:.2f}")
    print(f"Theoretical Cost (Single-Stage): ${theoretical_cost:.2f}")
    print(f"Savings Generated: ${savings:.2f} ({(savings/theoretical_cost)*100:.1f}%)")

def validate_ai_decisions(audit_decisions):
    print("\n🤖 AI ACCURACY SNAPSHOT")
    # Simple analysis of decision logs
    df = pd.DataFrame(audit_decisions)
    if not df.empty:
        print(df.groupby(['ai_risk_level', 'human_decision']).size())

# Execute the metrics
analyze_workflow_efficiency(processed_cases, approved_sars, rejected_cases, audit_decisions)
validate_ai_decisions(audit_decisions)


📊 WORKFLOW EFFICIENCY REPORT
Total Cases Screened (AI Stage 1): 5
Human Approved (Proceeded to Stage 2): 5
Human Rejected (Stopped at Stage 1): 0
Approval Rate: 100.0%

💰 COST OPTIMIZATION
Actual API Cost (Two-Stage): $1.00
Theoretical Cost (Single-Stage): $1.00
Savings Generated: $0.00 (0.0%)

🤖 AI ACCURACY SNAPSHOT
ai_risk_level  human_decision
High           APPROVED          5
dtype: int64


## 🏁 Step 6: Complete System Demonstration

Test your complete system with comprehensive scenarios to validate production readiness.

In [ ]:
# TODO: Run Complete System Test
# Students: Demonstrate your complete SAR processing system

def demonstrate_complete_system():
    print("🚀 DEMONSTRATION MODE INITIALIZED")
    
    # 1. Load
    c, a, t = load_and_preprocess_data()
    
    # 2. Screening (Grab top 3 for quick demo)
    selected = screen_high_risk_customers(c, a, t, top_n=3)
    
    if not selected:
        print("⚠️ Nenhum cliente selecionado para demonstração.")
        return

    # 3. Execute
    print("\n--- Processing Batch ---")
    p, app, rej, audit = run_two_stage_sar_workflow(selected)
    
    # 4. Report
    analyze_workflow_efficiency(p, app, rej, audit)
    
    print("\n🏁 System Demonstration Complete.")

# Execute Demo
demonstrate_complete_system()

🚀 DEMONSTRATION MODE INITIALIZED
📊 Loading Financial Data...
📈 Loaded: 150 customers, 178 accounts, 4268 transactions
🔍 Customer Risk Screening...
📊 Selected 3 high-risk customers for detailed AI analysis.

--- Processing Batch ---
🤖 Starting Two-Stage SAR Processing Workflow...

[1/3] Processing Customer: Jacqueline Rodriguez (ID: CUST_0111)
🔍 Running Risk Analyst Agent...
❌ Error processing case: Failed to parse Risk Analyst JSON output

[2/3] Processing Customer: Michael Stanley (ID: CUST_0062)
🔍 Running Risk Analyst Agent...


Traceback (most recent call last):
  File "c:\Projects\Agentic AI For Financial Services\1. Prompting for LLM Reasoning and Planning for Financial Services\Project - Automated SAR Detection & Reporting System\starter\src\risk_analyst_agent.py", line 216, in analyze_case
    result = RiskAnalystOutput(
  File "C:\Users\eduar\AppData\Roaming\Python\Python310\site-packages\pydantic\main.py", line 250, in __init__
    validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
pydantic_core._pydantic_core.ValidationError: 1 validation error for RiskAnalystOutput
classification
  Input should be 'Structuring', 'Sanctions', 'Fraud', 'Money_Laundering' or 'Other' [type=literal_error, input_value='Money_Laudering', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/literal_error

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\eduar\AppData\Local\Temp\ipykernel_


--- AI FINDINGS ---
Classification: Money_Laundering
Risk Level:     High
Confidence:     0.85
Reasoning:      The customer has a medium risk rating and has performed a large incoming wire transfer followed by multiple outgoing wire transfers in a short period ...
✅ Approved. Proceeding to Compliance Narrative generation.
🔍 DEBUG: Starting narrative generation...
📄 SAR Generated: SAR-F43A4ED8

[3/3] Processing Customer: Cindy Clayton (ID: CUST_0057)
🔍 Running Risk Analyst Agent...

--- AI FINDINGS ---
Classification: Structuring
Risk Level:     High
Confidence:     0.85
Reasoning:      The customer has a high risk rating and has been involved in numerous transactions, many of which are close to the $5,000 threshold. This suggests a p...
✅ Approved. Proceeding to Compliance Narrative generation.
🔍 DEBUG: Starting narrative generation...
📄 SAR Generated: SAR-1933A29A

📊 WORKFLOW EFFICIENCY REPORT
Total Cases Screened (AI Stage 1): 3
Human Approved (Proceeded to Stage 2): 2
Human Rejecte

## 📝 Implementation Checklist

### ✅ Workflow Integration Deliverables
- [X] **Data Loading**: Load and preprocess CSV data with proper error handling
- [X] **Customer Screening**: Implement risk-based screening to identify high-risk cases
- [X] **Two-Stage Workflow**: Build complete Risk Analyst → Human Gate → Compliance Officer flow
- [X] **Human Decision Gates**: Implement interactive approval/rejection points
- [X] **SAR Document Generation**: Create complete FinCEN-ready documents with metadata
- [X] **Audit Trail Creation**: Log all decisions and reasoning for regulatory examination
- [X] **Efficiency Metrics**: Calculate cost optimization and processing efficiency
- [X] **System Demonstration**: Test complete workflow with multiple scenarios

### ✅ Testing and Validation Requirements
- [X] **Component Validation**: Verify all foundation components and agents are available
- [X] **Integration Testing**: Run comprehensive test suites for all components with proper sys.path setup
- [X] **End-to-End Testing**: Test complete workflow with automated scenarios
- [X] **Error Handling Testing**: Validate graceful handling of edge cases and failures
- [X] **Output Validation**: Ensure SAR documents meet regulatory standards
- [X] **Performance Testing**: Measure workflow efficiency and processing times

### ✅ Technical Requirements
- [X] **Error Handling**: Robust exception handling for all workflow steps
- [X] **Data Validation**: Proper validation of all inputs and outputs
- [X] **File Management**: Organize outputs in appropriate directories
- [X] **Logging**: Comprehensive audit logging for compliance
- [X] **Performance**: Efficient processing of multiple cases
- [X] **User Experience**: Clear prompts and feedback for human reviewers
- [X] **Test Infrastructure**: Proper test imports and sys.path configuration

### ✅ Business Requirements  
- [X] **Regulatory Compliance**: Ensure all SAR documents meet FinCEN requirements
- [X] **Cost Optimization**: Demonstrate savings from two-stage processing
- [X] **Audit Readiness**: Create examination-ready documentation
- [X] **Quality Assurance**: Validate AI decisions with human oversight
- [X] **Scalability**: Design for processing larger datasets
- [X] **Production Readiness**: Complete testing validates system reliability

## 🎯 Success Criteria

By completion, your integrated system should:
- ✅ Process real financial data with proper validation
- ✅ Execute complete two-stage AI workflow with human gates
- ✅ Generate regulatory-compliant SAR documents
- ✅ Create comprehensive audit trails for all decisions
- ✅ Demonstrate measurable cost optimization benefits
- ✅ Handle errors gracefully and provide clear user feedback
- ✅ Pass all integration and end-to-end tests
- ✅ Meet production-ready quality standards

## 🚀 Next Steps

1. **Complete Implementation**: Fill in all TODO sections with working code
2. **Run Integration Tests**: Validate all components work together properly
3. **Execute End-to-End Tests**: Test complete workflow with automated scenarios
4. **Test Thoroughly**: Run complete workflow with various manual scenarios
5. **Validate Outputs**: Ensure SAR documents meet regulatory requirements
6. **Document Results**: Create final project documentation and metrics
7. **Prepare Presentation**: Demonstrate your system's capabilities and business value

**Congratulations on building a complete AI-powered SAR processing system! 🎉**

## 🧪 Step 7: Workflow Testing and Validation

Before finalizing your implementation, validate your complete system with comprehensive testing.

In [11]:
# 🧪 Workflow Integration Testing
# Validate your complete system with integration tests

import sys
import os

# Add tests directory to Python path for importing test modules
project_root = os.path.abspath('..')
tests_path = os.path.join(project_root, 'tests')
if tests_path not in sys.path:
    sys.path.insert(0, tests_path)

print(f"📁 Added tests directory to Python path: {tests_path}")

def run_integration_tests():
    """
    Run comprehensive integration tests to validate the complete workflow
    
    Tests include:
    1. Foundation components integration
    2. Agent communication and data flow
    3. End-to-end workflow execution
    4. Error handling and edge cases
    5. Output validation and compliance
    """
    print("🧪 Comprehensive Integration Testing")
    print("📋 TODO: Uncomment and run after implementing complete workflow")
    
    # Uncomment when your complete system is ready:
    try:
        # Import all test modules
        from test_foundation import TestCustomerData, TestAccountData, TestTransactionData, TestCaseData
        from test_risk_analyst import TestRiskAnalystAgent
        from test_compliance_officer import TestComplianceOfficerAgent
        import pytest
        
        pytest_flags = [
            "-v", 
            "--tb=short",
            "-p", "no:cacheprovider",
            "-W", "ignore::pytest.PytestAssertRewriteWarning"
        ]

        print("🔍 Running Foundation Component Tests...")
        foundation_result = pytest.main([
            f"{tests_path}/test_foundation.py", 
            "-v", 
            "--tb=short"
        ] + pytest_flags)
        
        print("🔍 Running Risk Analyst Agent Tests...")
        risk_result = pytest.main([
            f"{tests_path}/test_risk_analyst.py", 
            "-v", 
            "--tb=short"
        ] + pytest_flags)
        
        print("📝 Running Compliance Officer Agent Tests...")
        compliance_result = pytest.main([
            f"{tests_path}/test_compliance_officer.py", 
            "-v", 
            "--tb=short"
        ] + pytest_flags)
        
        # Calculate overall test results
        all_passed = foundation_result == 0 and risk_result == 0 and compliance_result == 0
        
        print("\n" + "="*60)
        print("📊 INTEGRATION TEST RESULTS:")
        print(f"   Foundation Components: {'✅ PASS' if foundation_result == 0 else '❌ FAIL'}")
        print(f"   Risk Analyst Agent: {'✅ PASS' if risk_result == 0 else '❌ FAIL'}")
        print(f"   Compliance Officer Agent: {'✅ PASS' if compliance_result == 0 else '❌ FAIL'}")
        print(f"   Overall Status: {'✅ ALL TESTS PASSED' if all_passed else '❌ SOME TESTS FAILED'}")
        
        if all_passed:
            print("\n🎉 Your system is ready for production workflow testing!")
            print("📝 Proceed to run the complete system demonstration.")
        else:
            print("\n⚠️ Fix failing tests before running the complete workflow.")
            print("📝 Return to previous notebooks to fix component issues.")
        
        return all_passed
            
    except ImportError as e:
        print(f"❌ Import Error: {e}")
        print("💡 Make sure all components are implemented:")
        print("   • foundation_sar.py")
        print("   • risk_analyst_agent.py") 
        print("   • compliance_officer_agent.py")
        return False

def validate_workflow_components():
    """Validate that all required components are available for integration"""
    print("🔍 Validating Workflow Components")
    
    components_status = {
        'foundation_sar': False,
        'risk_analyst_agent': False,
        'compliance_officer_agent': False,
        'test_modules': False
    }
    
    try:
        # Check foundation components
        from foundation_sar import CustomerData, CaseData, ExplainabilityLogger, DataLoader
        components_status['foundation_sar'] = True
        print("✅ Foundation components available")
    except ImportError:
        print("❌ Foundation components not available")
    
    try:
        # Check risk analyst agent
        from risk_analyst_agent import RiskAnalystAgent
        components_status['risk_analyst_agent'] = True
        print("✅ Risk Analyst Agent available")
    except ImportError:
        print("❌ Risk Analyst Agent not available")
    
    try:
        # Check compliance officer agent
        from compliance_officer_agent import ComplianceOfficerAgent
        components_status['compliance_officer_agent'] = True
        print("✅ Compliance Officer Agent available")
    except ImportError:
        print("❌ Compliance Officer Agent not available")
    
    try:
        # Check test modules
        from test_foundation import TestCustomerData
        from test_risk_analyst import TestRiskAnalystAgent  
        from test_compliance_officer import TestComplianceOfficerAgent
        components_status['test_modules'] = True
        print("✅ Test modules available")
    except ImportError:
        print("❌ Test modules not available")
    
    all_ready = all(components_status.values())
    
    print(f"\n📊 Component Status: {'✅ ALL READY' if all_ready else '⚠️ INCOMPLETE'}")
    if not all_ready:
        print("💡 Complete missing components before running integration tests")
    
    return all_ready

# Run component validation
components_ready = validate_workflow_components()

# Run integration tests if components are ready
if components_ready:
    print("\n🚀 All components ready - you can run integration tests!")
    run_integration_tests()
else:
    print("\n📋 Complete component implementation first, then run integration tests")

📁 Added tests directory to Python path: c:\Projects\Agentic AI For Financial Services\1. Prompting for LLM Reasoning and Planning for Financial Services\Project - Automated SAR Detection & Reporting System\starter\tests
🔍 Validating Workflow Components
✅ Foundation components available
✅ Risk Analyst Agent available
✅ Compliance Officer Agent available
✅ Test modules available

📊 Component Status: ✅ ALL READY

🚀 All components ready - you can run integration tests!
🧪 Comprehensive Integration Testing
📋 TODO: Uncomment and run after implementing complete workflow
🔍 Running Foundation Component Tests...
============================= test session starts =============================
platform win32 -- Python 3.10.11, pytest-9.0.2, pluggy-1.6.0 -- c:\Program Files\Python310\python.exe
rootdir: c:\Projects\Agentic AI For Financial Services\1. Prompting for LLM Reasoning and Planning for Financial Services\Project - Automated SAR Detection & Reporting System\starter
plugins: anyio-4.11.0, lan

In [12]:
# 🎯 End-to-End Workflow Testing
# Test the complete workflow with known test scenarios

def test_complete_workflow():
    """
    Test the complete SAR processing workflow end-to-end
    
    This test should:
    1. Load test data (can use a subset of actual data)
    2. Run customer screening
    3. Execute two-stage AI analysis
    4. Simulate human decisions (automated for testing)
    5. Generate SAR documents
    6. Validate all outputs
    7. Check audit trail completeness
    """
    print("🎯 End-to-End Workflow Testing")
    print("📋 TODO: Implement after completing all workflow components")
    
    # Example end-to-end test structure (uncomment when ready):
    try:
        print("🚀 Starting end-to-end workflow test...")
        
        # Test data preparation
        print("📊 Loading test data...")
        customers_data, accounts_data, transactions_data = load_and_preprocess_data()
        
        if not customers_data:
            print("⚠️ No test data available - implement data loading first")
            return False
        
        # Test customer screening
        print("🔍 Testing customer screening...")
        selected_customers = screen_high_risk_customers(
            customers_data, accounts_data, transactions_data, top_n=2
        )
        
        if not selected_customers:
            print("⚠️ No customers selected - check screening criteria")
            return False
        
        print(f"✅ Selected {len(selected_customers)} customers for testing")
        
        # Test workflow with automated decisions (for testing)
        print("🤖 Testing automated workflow...")
        test_results = {
            'cases_processed': 0,
            'sars_generated': 0,
            'errors': []
        }
        
        for customer_data in selected_customers:
            try:
                # Create case
                loader = DataLoader(explainability_logger)
                case_data = loader.create_case_from_data(
                    customer_data['customer'],
                    customer_data['accounts'],
                    customer_data['transactions']
                )
                
                # Test Risk Analyst
                risk_analysis = risk_agent.analyze_case(case_data)
                assert hasattr(risk_analysis, 'classification'), "Risk analysis missing classification"
                assert hasattr(risk_analysis, 'confidence_score'), "Risk analysis missing confidence"
                
                # Test Compliance Officer (simulate approval)
                compliance_review = compliance_agent.generate_compliance_narrative(case_data, risk_analysis)
                assert hasattr(compliance_review, 'narrative'), "Compliance review missing narrative"
                
                # Test SAR generation
                sar_document = create_sar_document(case_data, risk_analysis, compliance_review)
                assert sar_document, "SAR document generation failed"
                
                test_results['cases_processed'] += 1
                test_results['sars_generated'] += 1
                
                print(f"✅ Successfully processed: {customer_data['customer']['name']}")
                
            except Exception as e:
                test_results['errors'].append(f"Error processing {customer_data['customer']['name']}: {e}")
                print(f"❌ Error processing: {customer_data['customer']['name']}: {e}")
        
        # Test results summary
        print("\n📊 END-TO-END TEST RESULTS:")
        print(f"   Cases Processed: {test_results['cases_processed']}")
        print(f"   SARs Generated: {test_results['sars_generated']}")
        print(f"   Errors: {len(test_results['errors'])}")
        
        if test_results['errors']:
            print("❌ Test Errors:")
            for error in test_results['errors']:
                print(f"     • {error}")
        
        success = len(test_results['errors']) == 0 and test_results['cases_processed'] > 0
        
        if success:
            print("\n🎉 END-TO-END TEST PASSED!")
            print("✅ Your complete workflow is ready for production use!")
        else:
            print("\n⚠️ END-TO-END TEST HAD ISSUES")
            print("📝 Fix the errors above before deploying to production")
        
        return success
        
    except Exception as e:
        print(f"❌ End-to-end test failed: {e}")
        print("💡 Ensure all components are properly implemented")
        return False

# Run end-to-end test
print("📋 TODO: Run end-to-end test after implementing complete workflow")
test_success = test_complete_workflow()

📋 TODO: Run end-to-end test after implementing complete workflow
🎯 End-to-End Workflow Testing
📋 TODO: Implement after completing all workflow components
🚀 Starting end-to-end workflow test...
📊 Loading test data...
📊 Loading Financial Data...
📈 Loaded: 150 customers, 178 accounts, 4268 transactions
🔍 Testing customer screening...
🔍 Customer Risk Screening...
📊 Selected 2 high-risk customers for detailed AI analysis.
✅ Selected 2 customers for testing
🤖 Testing automated workflow...
🔍 DEBUG: Starting narrative generation...
✅ Successfully processed: Jacqueline Rodriguez
🔍 DEBUG: Starting narrative generation...
✅ Successfully processed: Michael Stanley

📊 END-TO-END TEST RESULTS:
   Cases Processed: 2
   SARs Generated: 2
   Errors: 0

🎉 END-TO-END TEST PASSED!
✅ Your complete workflow is ready for production use!
